<a href="https://colab.research.google.com/github/lizjwh/groupIAI_5take2/blob/main/5%20Completely%20New%20Model/CommpletelyNewModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install torchvision and kaggle
!pip install torchvision
!pip install kaggle
!pip install tqdm
!pip install colorama

In [ ]:
#import  needed libraries and check the used gpu
import torch
from torch import nn,optim
from torchvision import transforms, models ,datasets
import numpy as np
import matplotlib.pyplot as plt
import glob
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
! export KAGGLE_USERNAME="emmet454" && export KAGGLE_KEY="ee00fbc0728a71f5c5f712029e3ef004" && kaggle datasets download --force --unzip emilyburt/intro-to-ai

In [ ]:
#lines to reorganise the folder system
def rename_folder(folder_path, new_name):
    """
    Rename a folder.

    Args:
    - folder_path (str): Path to the folder to be renamed.
    - new_name (str): New name for the folder.
    """
    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' does not exist.")
        return

    # Extract the directory path and folder name
    directory, old_name = os.path.split(folder_path)

    # Create the new path with the new folder name
    new_path = os.path.join(directory, new_name)

    try:
        # Rename the folder
        os.rename(folder_path, new_path)
        print(f"Folder '{folder_path}' renamed to '{new_path}'.")
    except OSError as e:
        print(f"Error: Failed to rename folder '{folder_path}' to '{new_path}': {e}")


folder_path = "/content/Dataset/Full Mobility/Left hand/Space"
new_name = "space"
rename_folder(folder_path, new_name)

folder_path = "/content/Dataset/Full Mobility/Right hand/Space"
rename_folder(folder_path, new_name)

folder_path = "/content/Dataset/Restricted mobility/Left hand/Space"
rename_folder(folder_path, new_name)

folder_path = "/content/Dataset/Restricted mobility/Right hand/Space"
rename_folder(folder_path, new_name)


#These functions allow each folder to be combined
from torch.utils.data import ConcatDataset

def create_dataset(root_dir):
    """
    Creates an ImageFolder dataset from the specified root directory.
    """
    data_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])
    dataset = datasets.ImageFolder(root_dir, transform=data_transforms)
    return dataset

def combine_datasets(root_folders):
    """
    Combines datasets from multiple root folders into one dataset.
    """
    datasets = [create_dataset(folder) for folder in root_folders]
    combined_dataset = ConcatDataset(datasets)

    # Define the desired order of classes
    class_order = ['A', 'B', 'C', 'D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','Space']   # Define your desired order
    # Extract class-to-index mappings from individual datasets and merge them
    class_to_idx = {}
    idx = -78
    for dataset in datasets:
        for class_name in class_order:
            if class_name in dataset.class_to_idx:
              if class_name == 'space':
                class_to_idx[class_name] = 28
                idx += 1
              else:
                class_to_idx[class_name] = idx
                idx += 1

    return combined_dataset, class_to_idx


def main():
    # Choose which folders to include in test here
    root_folders = ['/content/Dataset/Restricted mobility/Left hand',
                    '/content/Dataset/Restricted mobility/Right hand',
                    '/content/Dataset/Full Mobility/Left hand',
                    '/content/Dataset/Full Mobility/Right hand']

    combined_dataset, combined_class_to_idx = combine_datasets(root_folders)

    print(f"Class to index mapping: {combined_class_to_idx}")
    return combined_dataset, combined_class_to_idx

combined_dataset, combined_dataset.class_to_idx = main()



#define the transform for the dataset
#test_transforms = transforms.Compose([transforms.Resize((224,224)),
#                                      transforms.ToTensor(),
#                                     transforms.Normalize([0.485, 0.456, 0.406],
 #                                                           [0.229, 0.224, 0.225])])

#transform the data
#test_data = datasets.ImageFolder(path,test_transforms)
#path = '/content/Dataset/Restricted mobility/Right hand'
#test_data.append(datasets.ImageFolder(path,test_transforms))
#path = '/content/Dataset/Full Mobility/Left hand'
#test_data.append(datasets.ImageFolder(path,test_transforms))
#path = '/content/Dataset/Full Mobility/Right hand'
#test_data.append(datasets.ImageFolder(path,test_transforms))
#print(f"class to index mapping: {test_data.class_to_idx}")

#load some of the test data
testloader = torch.utils.data.DataLoader(combined_dataset, batch_size=50, shuffle=True)
images, labels = next(iter(testloader))
print(labels)
#print(images.size())

In [ ]:
# Define transforms for the training data and testing data
train_path='asl_alphabet_train'
valid_path='asl_alphabet_valid'

train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.RandomRotation(30),
                                       transforms.RandomHorizontalFlip(p=0.3),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

In [ ]:
# load data to loaders
train_data = datasets.ImageFolder(train_path, transform=train_transforms)
test_data = datasets.ImageFolder(valid_path, transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=512)

#print used Device
print(f"Device used: {torch.cuda.get_device_name(0)}")

#print class to index mapping
print(f"class to index mapping: {train_data.class_to_idx}")

#Now we can create our new model with Squeezenet1_1

In [ ]:
model = models.squeezenet1_1(pretrained=True)